**Importing libraries**

In [33]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

**open csv file and seprate features and lables**

In [34]:
df= pd.read_csv("/content/Car details v3.csv")

# from rows whose mileage in km/kg 
for i in range(len(df)):
  if str(df.loc[i,'mileage']).endswith("km/kg"):
    df.loc[i,'mileage']=np.nan
    
df = df.dropna()

df['year']=2021-df['year']

df['name'] = df['name'].str.strip()

df['mileage'] = df['mileage'].str.slice(0, -5)
df['mileage'] = df['mileage'].str.strip()

df['engine'] = df['engine'].str.slice(0, -2)
df['engine'] = df['engine'].str.strip()

df['max_power'] = df['max_power'].str.slice(0, -3)
df['max_power'] = df['max_power'].str.strip()

df['max_power']=pd.to_numeric(df['max_power'])
df['engine']= pd.to_numeric(df['engine'])
df['mileage'] = pd.to_numeric(df['mileage'])

X= df.iloc[:,[1,3,4,5,6,7,8,9,10,12]].values
Y= df.iloc[:,2].values
Y = Y.reshape(-1,1)


**Dealing with empty values**

In [35]:
#As we already removed rows with empty values so no need for this
#imputer = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
#X = imputer.fit_transform(X)
#Y = imputer.fit_transform(Y)

**perform encoding ie to convert all to numerical datas**

In [36]:
le2 = LabelEncoder()
X[:,2] =  le2.fit_transform(X[:,2])
le3 = LabelEncoder()
X[:,3] =  le3.fit_transform(X[:,3])
le4 = LabelEncoder()
X[:,4] =  le4.fit_transform(X[:,4])
le5 = LabelEncoder()
X[:,5] =  le5.fit_transform(X[:,5])

print(X[0,:])

[7 145500 0 1 1 0 23.4 1248 74.0 5.0]


**feature scaling**

In [37]:
#sc= StandardScaler()
#X = sc.fit_transform(X)
#Y = sc.fit_transform(Y)

**splitting test and train data**

In [38]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y,test_size=0.2,random_state = 0)

**training model**

In [39]:
randForestReg = RandomForestRegressor()
### selecting parameters
n_estimators = [int(x) for x in np.linspace(start=100 , stop = 1200 , num = 12)]
max_features = ['auto','sqrt']
max_depth = [int(x) for x in np.linspace(5,30,num = 6)]
min_samples_split = [2,5,10,15,100]
min_samples_leaf = [1,2,5,10]

random_grid = {'n_estimators': n_estimators ,'max_features': max_features , 'max_depth': max_depth ,'min_samples_split' : min_samples_split,
              'min_samples_leaf': min_samples_leaf }

rf_random = RandomizedSearchCV(estimator = randForestReg , param_distributions = random_grid , scoring = 'neg_mean_squared_error',
                               n_iter = 10,cv=5,random_state = 42 , n_jobs = 1)

rf_random.fit(X_train,Y_train.ravel())

y_predicted = rf_random.predict(X_test)

y_predicted = y_predicted.reshape(-1,1)


**analyse model**

In [40]:
#print((y_predicted))
#print((Y_test))
  
print(mean_absolute_error((Y_test),(y_predicted)))

70626.29506957968


**importing model to pickle file**

In [41]:
import pickle

file = open('/car_price_model.pkl','wb')

pickle.dump(rf_random,file)
